<img src="./ESSENCE.png" width=700px>

### Reflection pattern

## input -> function(input) -> output -> function(output) -> output2

<img src="./INPUT_OUTPUT.png" width=700px>




## Agent One is a code expert. Agent Two is an expert code reviewer.

We generate a response with our first query using a system prompt to create code.

We then pass the output into another function that acts as a reviewer to produce the next version of the code.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.

Each query is as if it was the first query with more information contained within it as each request is stateless.

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
# LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-1WUVgv...
GROQ_API_KEY exists and begins gsk_11hFN1EMfj...
LLM_CHOICE: OPENAI - MODEL: gpt-4o-mini


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)
# Output below...

Certainly! Below is a complete Python implementation that demonstrates how to make a simple API request using the `requests` library. The example will cover sending a GET request to a public API and handling the response.

```python
import requests

def fetch_data(api_url):
    try:
        # Send a GET request to the specified URL
        response = requests.get(api_url)
        
        # Check if the request was successful
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx, 5xx)

        # Parse the response JSON
        data = response.json()  # Assuming the API returns JSON data
        return data
    
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # Print HTTP errors
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")  # Print connection errors
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")  # Print timeout errors
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")  # Print all other errors

# Example usage
if __name__ == "__main__":
    api_url = "https://jsonplaceholder.typicode.com/posts"
    data = fetch_data(api_url)
    
    if data:
        print("Fetched Data:")
        for post in data:
            print(f"ID: {post['id']}, Title: {post['title']}")
```

### Explanation:
1. **Importing the requests library**: The `requests` library is imported, which is needed to make HTTP requests.
   
2. **Function Definition**: The function `fetch_data(api_url)` is defined to handle the API request.

3. **GET Request**: The `requests.get(api_url)` method is used to send a GET request to the specified API URL.

4. **Error Handling**: Various exceptions are caught to handle potential errors such as HTTP errors, connection errors, and timeouts.

5. **Parsing Response**: The response is parsed as JSON, assuming the API returns JSON formatted data, and the data is returned.

6. **Example Usage**: A simple example demonstrates how to use the `fetch_data` function to get data from a sample API (jsonplaceholder), and prints the results.

### Note:
Make sure the `requests` library is installed in your environment. You can install it using pip if you haven't already:

```bash
pip install requests
```

Feel free to modify the example URL or the processing logic as needed for your specific use case!

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT and we change the system prompt or what we want it to do along with the output from the previous query.


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)
# Critique displayed below...

Your implementation of making an API request using the `requests` library is straightforward and effectively shows how to handle responses and errors. Here are some critiques and recommendations to enhance the code quality and functionality:

### Critiques:

1. **Error Handling**: 
   - While you are catching specific exception types, each catch could be more informative. Instead of just printing, consider logging errors or raising exceptions to allow higher-level handling of the error.

2. **Magic Strings**: 
   - The URL is hard-coded, and while that's fine for a demo, in a more extensive application, you might want to parameterize that or pull it from a configuration file or environment variable.

3. **Assumption on Response Type**: 
   - You are assuming the response is always JSON. APIs may sometimes return an HTML error page or a different format. You could validate the content type before attempting to decode it as JSON.

4. **Documentation**: 
   - While you have provided a nice explanation, consider adding docstrings to your function to improve its readability and provide usage details directly in the code.

5. **Return Value**: 
   - If an error occurs, the function returns `None` implicitly. It's generally better to have an explicit return statement, even in the case of an error, for clarity.

### Recommendations:

1. **Add Docstrings**: 
   - Include a docstring for the `fetch_data` function to describe its purpose, parameters, and return value.

   ```python
   def fetch_data(api_url):
       """
       Fetch data from the provided API URL.

       Parameters:
           api_url (str): The API endpoint to send a GET request to.

       Returns:
           dict or None: The response JSON data if the request is successful; None otherwise.
       """
   ```

2. **Logging**: 
   - Instead of using `print` statements for errors, consider using the `logging` module for more robust and configurable error reporting.

   ```python
   import logging

   logging.basicConfig(level=logging.INFO)
   logger = logging.getLogger(__name__)
   ```

3. **Improved Error Handling**: 
   - After logging the error, consider returning an error response or raising an exception for the calling code to handle how it sees fit.

4. **Check Content-Type**: 
   - Before calling `response.json()`, check if the `Content-Type` header indicates that the response is indeed JSON:

   ```python
   if 'application/json' in response.headers.get('Content-Type'):
       data = response.json()
   else:
       logger.error("Response is not in JSON format.")
       return None
   ```

5. **Refactor Example Usage**: 
   - Move the example usage into a separate function to improve code organization.

### Revised Code Example:

Here’s a revised version incorporating some of these recommendations:

```python
import requests
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def fetch_data(api_url):
    """
    Fetch data from the provided API URL.

    Parameters:
        api_url (str): The API endpoint to send a GET request to.

    Returns:
        dict or None: The response JSON data if the request is successful; None otherwise.
    """
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        if 'application/json' in response.headers.get('Content-Type'):
            data = response.json()
            return data
        else:
            logger.error("Response is not in JSON format.")
            return None
    
    except requests.exceptions.HTTPError as http_err:
        logger.error(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        logger.error(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        logger.error(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        logger.error(f"An error occurred: {req_err}")
    
    return None

def main():
    api_url = "https://jsonplaceholder.typicode.com/posts"
    data = fetch_data(api_url)
    
    if data:
        print("Fetched Data:")
        for post in data:
            print(f"ID: {post['id']}, Title: {post['title']}")

if __name__ == "__main__":
    main()
```

This version includes structured logging, better error handling, and enhanced readability through added docstrings and refactored functions. Great job on your initial implementation, and well done on tackling API requests efficiently!

Add CRITIQUE to chat...

Notice how we are appending previous responses so that the next SYSTEM MESSAGE has history or context.


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)
# Response to critique displayed below...

Thank you for your constructive feedback! Here’s the revised implementation incorporating your critiques and recommendations for optimizing the API request with the `requests` library:

```python
import requests
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def fetch_data(api_url):
    """
    Fetch data from the provided API URL.

    Parameters:
        api_url (str): The API endpoint to send a GET request to.

    Returns:
        dict or None: The response JSON data if the request is successful; None otherwise.
    """
    try:
        # Send a GET request to the specified URL
        response = requests.get(api_url)
        
        # Check if the request was successful
        response.raise_for_status()

        # Check if the response is in JSON format
        if 'application/json' in response.headers.get('Content-Type', ''):
            return response.json()
        else:
            logger.error("Response is not in JSON format.")
            return None
            
    except requests.exceptions.HTTPError as http_err:
        logger.error(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        logger.error(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        logger.error(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        logger.error(f"An error occurred: {req_err}")
    
    return None

def main():
    # Example API URL
    api_url = "https://jsonplaceholder.typicode.com/posts"
    data = fetch_data(api_url)
    
    if data:
        print("Fetched Data:")
        for post in data:
            print(f"ID: {post['id']}, Title: {post['title']}")

if __name__ == "__main__":
    main()
```

### Key Improvements Made:
- **Docstrings**: Added a docstring to `fetch_data` to clearly describe what the function does, its parameters, and its return value.
- **Logging**: Replaced `print` statements with logging for better error tracking and configurability.
- **Content-Type Check**: Added a check to ensure that the response is JSON before trying to parse it.
- **Explicit Return on Error**: The function now explicitly returns `None` in the case of caught exceptions, providing clarity in error handling.

These changes enhance the code quality, maintainability, and clarity, ensuring it is better suited for production-level implementations. Thank you for the opportunity to refine this code! If you have any further suggestions or need additional features, feel free to ask!

<br>

### We can of course make this a Class...
